In [2]:
import pandas as pd
import os
import pickle

## Download and prepare the IEDB Population coverage tool

1. Download the population coverage tool from [here](http://tools.iedb.org/population/download/).
2. Untar the contents of the downloaded file to the root folder of this notebook.
3. The root folder of this notebook will contain the `population_coverage` folder.
4. Run the cell below to make sure you have a working version of the population coverage tool (you should see the usage output).



In [4]:
os.system("python ./population_coverage/calculate_population_coverage.py --help")

usage: python calculate_population_coverage.py [-h] -p [POPULATION] -c [MHC_CLASS] -f [FILE]

Created on: 03/06/2017 @author: Dorjee Gyaltsen @brief: calculates population
coverage - standalone version

options:
  -h, --help            show this help message and exit
  --list                list all population and ethnicity
  --plot PATH           generate a plot.
  --version             show program's version number and exit

required arguments:
  -p POPULATION [POPULATION ...], --population POPULATION [POPULATION ...]
                        select comma-separated area(s) or population(s)
  -c MHC_CLASS [MHC_CLASS ...], --mhc_class MHC_CLASS [MHC_CLASS ...]
                        select one or more comma-separated mhc class option -
                        I, II, combined
  -f FILE, --file FILE  a file containing a list of epitopes and associated
                        alleles (comma-separated)


0

####  Format the AFND frequencies in a pickle to be used by the IEDB pop coverage

In [5]:
afnd_df = pd.read_csv("AFND_data_locus_all.csv")
afnd_df.Population.unique()
afnd_df["Population"] = afnd_df["Population"].apply(lambda x: x.replace("á", "a")) 
afnd_df.to_csv("AFND_data_locus_all.csv")
from collections import OrderedDict
full_dict = OrderedDict()
for population in afnd_df.Population.unique():
    pop_dict = OrderedDict()
    for locus in afnd_df.Locus.unique():
        tmp_df = afnd_df[(afnd_df.Population==population) & (afnd_df.Locus==locus)]
        freqs = list(tmp_df.apply(lambda x: ("HLA-"+x.Allele, x["Allele Frequency"]), axis=1))
        pop_dict["HLA-{}".format(locus)] = freqs
    full_dict[population] = pop_dict
final_dict = {"I": full_dict}
with open("our_frequencies_pickle.p", "wb") as pickle_file:
    pickle.dump(final_dict, pickle_file)

#### Move this file to ./population_coverage rename it to match and replace the old file

In [23]:
#in linux:
os.system("cp ./our_frequencies_pickle.p  ./population_coverage/population_coverage_pickle/population_genotype_map.p")

0

#### Fix the init file

In [35]:
init_file_content = '''
import pickle
from pkg_resources import resource_filename  # @UnresolvedImport

package_name = "population_coverage_pickle"
pickle_filename = "population_genotype_map.p"
pickle_file_path = resource_filename(package_name, pickle_filename)

with open(pickle_file_path, "rb") as pickle_file:
    population_coverage = pickle.load(pickle_file)
    #country_ethnicity = pickle.load(pickle_file)
    #ethnicity = pickle.load(pickle_file)
    country_ethnicity = population_coverage
    ethnicity = population_coverage
'''

init_file_location = "./population_coverage/population_coverage_pickle/__init__.py"

with open(init_file_location, "w") as tmp_file:
    tmp_file.write(init_file_content)

## Run population coverage for each of our populations

In [3]:
import os
from pathlib import Path
import pandas as pd
import plotly.express as px
from numpy import trapz
from scipy.spatial import distance
import numpy as np

In [11]:

def append_n_peptides(ideal_n_peptides, allele, ideal_dataset):
    ideal_n_peptides = int(ideal_n_peptides)
    peptides = range(ideal_n_peptides)
    allele = row.Allele
    alleles = list(np.repeat(allele, ideal_n_peptides))
    ideal_dataset["peptide"].extend(peptides)
    ideal_dataset["allele"].extend(alleles)
    return ideal_dataset

def run_population_coverage(population, afnd_df, dataset_name, exec_file, output_directory="."):
    
    # calculate for the real dataset
    print("===============================")
    print("Calculating PC for {} for dataset {}".format(population, dataset_name))
    print("===============================")
    root = Path(output_directory)
    print("Saving outputs to: {}".format(root))
    root.mkdir(parents=True, exist_ok=True)
    
    population_tmp_name = population.replace(" ", "_")
    population_tmp_name = population_tmp_name.replace("/", "_")
    dataset_real = os.path.join(Path(os.getcwd()), Path("./datasets/{}.csv".format(dataset_name)))
    dataset_real_pd = pd.read_csv(dataset_real, header=None, sep="\t")
    dataset_real_pd.columns = ["peptide", "allele", "locus"]
    dataset_real_pd["locus"] = dataset_real_pd["allele"].apply(lambda x: x[x.find("-")+1:x.find("*")])
    dataset_real_pd = dataset_real_pd[dataset_real_pd.locus.isin(["A", "B", "C"])]
    dataset_peptide_n = dataset_real_pd.shape[0]
    dataset_allele_n = dataset_real_pd["allele"].unique().shape[0]
    dataset_locus_n = dataset_real_pd["locus"].unique().shape[0]
    print(dataset_real_pd)
    print("Looking at dataset {} with total {} peptides across {} alleles {} loci".format(dataset_name, \
                                                                                 dataset_peptide_n,\
                                                                                 dataset_allele_n, \
                                                                                 dataset_locus_n))
    
    output_file_real = os.path.join(root, "{}_{}_real.txt".format(dataset_name, population_tmp_name))
    command = "python {} -p \"{}\" -c I -f {} > {}".format(exec_file, population, dataset_real, output_file_real)
    print("Running IEDB tool for population coverage and saving output to: {}".format(output_file_real))
    print(command)
    os.system(command)
    print(output_file_real)
    res_pd_real = pd.read_csv(output_file_real, skiprows=6, sep="\t")
    
    # calculate for the ideal dataset
    print("===============================")
    print("Calculating PC for {} for ideal dataset {}".format(population, dataset_name))
    print("===============================")
    tmp_afnd_df = afnd_df[afnd_df.Population == population]
    tmp_afnd_df = tmp_afnd_df[tmp_afnd_df.Locus.isin(["A", "B", "C"])]
    locus_n = len(tmp_afnd_df["Locus"].unique())
    allele_n = len(tmp_afnd_df["Allele"].unique())
    print("The original dataset {} has {} peptides".format(dataset_name, dataset_peptide_n))
    print("The population has {} alleles across {} loci".format(allele_n, locus_n))
    tmp_afnd_df["ideal_n_peptides"] = tmp_afnd_df["Allele Frequency"].apply(lambda x: int((dataset_peptide_n * x)/locus_n))
    tmp_afnd_df = tmp_afnd_df.sort_values(by=["Allele", "Locus"])
    tmp_afnd_df = tmp_afnd_df[tmp_afnd_df.ideal_n_peptides > 0]
    ideal_dataset_df = tmp_afnd_df.loc[tmp_afnd_df.index.repeat(tmp_afnd_df.ideal_n_peptides)].reset_index(drop=True)
    ideal_dataset_df["peptide"] = ideal_dataset_df.index
    ideal_dataset_df["peptide"] =  ideal_dataset_df["peptide"].apply(lambda x: "pep_{}".format(x))
    ideal_dataset_df["allele"] = ideal_dataset_df.Allele.apply(lambda x: "HLA-{}".format(x))
    ideal_dataset_df = ideal_dataset_df[["peptide", "allele"]]
    ideal_dataset_file = os.path.join(root, Path("./{}_{}_ideal.csv".format(dataset_name, population_tmp_name)))
    ideal_dataset_df.to_csv(ideal_dataset_file, header=None, index=False, sep="\t")
    print("Ideal dataset has {} peptides across {} alleles".format(len(ideal_dataset_df.peptide),
                                                                       len(ideal_dataset_df.allele.unique())))
    print(tmp_afnd_df[["Allele", "Locus", "ideal_n_peptides"]].reset_index())
    
    print("Saved dummy ideal dataset to: {}".format(ideal_dataset_file))
    output_file_ideal = os.path.join(root, "{}_{}_ideal.txt".format(dataset_name, population_tmp_name))
    print("Running IEDB tool for population coverage and saving output to: {}".format(output_file_ideal))
    command = "python {} -p \"{}\" -c I -f {} > {}".format(exec_file, population, ideal_dataset_file, output_file_ideal)
    os.system(command)
    
    print("===============================")
    print("Plotting results")
    print("===============================")
    #plot and calculate differences
    res_pd_ideal = pd.read_csv(output_file_ideal, skiprows=6, sep="\t")
    
    res_pd_ideal["type"] = res_pd_ideal.epitope_hits.apply(lambda x: "ideal")
    res_pd_real["type"] = res_pd_real.epitope_hits.apply(lambda x: "real")
    res_pd_all = pd.concat([res_pd_ideal, res_pd_real])
    results_output = os.path.join(root, "{}_{}_results.csv".format(dataset_name, population_tmp_name))
    res_pd_all.to_csv(results_output)
    
    fig = px.scatter(res_pd_all\
             , x="epitope_hits", y="cumulative_coverage", color="type")
    #fig.show()
    results_fig_output = os.path.join(root, "{}_{}_plot.svg".format(dataset_name, population_tmp_name))
    fig.write_image(results_fig_output, width=1500, height=500)

    print("===============================")
    print("Calculating JS divergence")
    print("===============================")
    # Compute JS-divergence
    max_y = max(res_pd_real.epitope_hits.max(), res_pd_ideal.epitope_hits.max())
    y_real = list(res_pd_real.cumulative_coverage)
    y_ideal = list(res_pd_ideal.cumulative_coverage)
    max_y_len = max(len(y_real), len(y_ideal))
    y_real_fit = [y_real[i] if i<len(y_real) else 0 for i in range(max_y_len)]
    y_ideal_fit = [y_ideal[i] if i<len(y_ideal) else 0 for i in range(max_y_len)] 
    dist = distance.jensenshannon(y_real_fit, y_ideal_fit)
    print("JS divergence: {}".format(dist))
    return dist
    

### Test for single population

In [12]:
from pathlib import Path

out_dir = "./iedb_afnd_pc_results"
exec_file = Path("./population_coverage/calculate_population_coverage.py")
tmp_res = run_population_coverage("American Samoa", afnd_df, "mhcflurry_peptides_ba", exec_file, ".")

Calculating PC for American Samoa for dataset mhcflurry_peptides_ba
Saving outputs to: .
           peptide       allele locus
0        AADFPGIAR  HLA-A*01:01     A
1       AADKAAAAAY  HLA-A*01:01     A
2        AADKAAAAY  HLA-A*01:01     A
3        AADSFATSY  HLA-A*01:01     A
4       AAELDRTEEY  HLA-A*01:01     A
...            ...          ...   ...
165527   YVYFYDLSY  HLA-C*15:02     C
165528   YWMGGTTYF  HLA-C*15:02     C
165529   YYFSYPLFV  HLA-C*15:02     C
165530   YYGRWVHEF  HLA-C*15:02     C
165531   YYKKTFSAL  HLA-C*15:02     C

[165532 rows x 3 columns]
Looking at dataset mhcflurry_peptides_ba with total 165532 peptides across 131 alleles 3 loci
Running IEDB tool for population coverage and saving output to: ./mhcflurry_peptides_ba_American_Samoa_real.txt
python population_coverage/calculate_population_coverage.py -p "American Samoa" -c I -f /home/anja/Documents/HLAequity/HLAlleleBias/Datasets_population_coverage/datasets/mhcflurry_peptides_ba.csv > ./mhcflurry_peptides_ba_

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


### Run for all populations

In [7]:
import numpy as np
# - if you want to restart from middle
#all_populations = afnd_df.Population.unique()
#np.argwhere(all_populations=='Russia Bering Island Aleuts')
#all_populations = all_populations[166:]

In [58]:
all_populations = afnd_df.Population.unique()
all_datasets = ["mhcflurry_peptides_ba", "netmhc_peptides_ba"]
out_dir = "./iedb_afnd_pc_results"
exec_file = Path("./population_coverage/calculate_population_coverage.py")

results = {"population": [], "dataset": [], "js_div": []}
failed = {"population": [], "dataset": []}
for population in all_populations:
    for dataset in all_datasets:
        try:
            res = run_population_coverage(population, afnd_df, dataset, exec_file, out_dir)
            results["population"].append(population)
            results["dataset"].append(dataset)
            results["js_div"].append(res)
        except:
            print("Failed for {}".format(population))
            failed["population"].append(population)
            failed["dataset"].append(dataset)
            
pd.DataFrame(results).to_csv(out_dir+"/all_results.csv")

Calculating PC for American Samoa for dataset mhcflurry
Saving outputs to: iedb_afnd_pc_results
           peptide       allele locus
0        AADFPGIAR  HLA-A*01:01     A
1       AADKAAAAAY  HLA-A*01:01     A
2        AADKAAAAY  HLA-A*01:01     A
3        AADSFATSY  HLA-A*01:01     A
4       AAELDRTEEY  HLA-A*01:01     A
...            ...          ...   ...
187590   YVYFYDLSY  HLA-C*15:02     C
187591   YWMGGTTYF  HLA-C*15:02     C
187592   YYFSYPLFV  HLA-C*15:02     C
187593   YYGRWVHEF  HLA-C*15:02     C
187594   YYKKTFSAL  HLA-C*15:02     C

[187485 rows x 3 columns]
Looking at dataset mhcflurry with total 187485 peptides across 132 alleles 3 loci
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\mhcflurry_American_Samoa_real.txt
python IEDB_Population_Coverage-3.0.2\population_coverage\calculate_population_coverage.py -p "American Samoa" -c I -f C:\Users\ac121\Documents\HLAequity\HLAlleleBias\Datasets_coverage\datasets\mhcflurry_peptides.csv > i

Plotting results
Calculating JS divergence
JS divergence: 0.5032643364599204
Calculating PC for American Samoa for dataset hlathena
Saving outputs to: iedb_afnd_pc_results
             peptide       allele locus
0         AIDNSRNILY  HLA-A*01:01     A
1       AIGTYLDQYIVY  HLA-A*01:01     A
2          ALDDFTICY  HLA-A*01:01     A
3         ALDDFTICYF  HLA-A*01:01     A
4         ALDENGNLWY  HLA-A*01:01     A
...              ...          ...   ...
142295     YTSRIVVRL  HLA-C*17:01     C
142296     YVAPKAVWL  HLA-C*17:01     C
142297     YVAPRRILL  HLA-C*17:01     C
142298     YVVPYTIHL  HLA-C*17:01     C
142299     YVYKNVSQL  HLA-C*17:01     C

[142300 rows x 3 columns]
Looking at dataset hlathena with total 142300 peptides across 92 alleles 3 loci
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\hlathena_American_Samoa_real.txt
python IEDB_Population_Coverage-3.0.2\population_coverage\calculate_population_coverage.py -p "American Samoa" -c I -f C:\U

Plotting results
Calculating JS divergence
JS divergence: 0.4480874451587635
Calculating PC for Australia Cape York Peninsula Aborigine for dataset netmhc
Saving outputs to: iedb_afnd_pc_results
           peptide       allele locus
0        AAAAFEAAL  HLA-B*48:01     B
1        AAAAFEAAL  HLA-C*14:02     C
2        AAADFAHAE  HLA-B*44:03     B
3        AAAKTPVIV  HLA-A*02:01     A
4        AAAKTPVIV  HLA-A*68:02     A
...            ...          ...   ...
170465  YDGRYWTMWK  HLA-C*08:03     C
170466   YEPSQSAQL  HLA-B*55:01     B
170467    YKKEQTLK  HLA-B*55:01     B
170468  YLRMRRTAAL  HLA-B*15:10     B
170469    YVALDFEQ  HLA-C*08:03     C

[170107 rows x 3 columns]
Looking at dataset netmhc with total 170107 peptides across 109 alleles 3 loci
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\netmhc_Australia_Cape_York_Peninsula_Aborigine_real.txt
python IEDB_Population_Coverage-3.0.2\population_coverage\calculate_population_coverage.py -p "Austral

Calculating JS divergence
JS divergence: 0.6051816478218356
Calculating PC for Australia Groote Eylandt Aborigine for dataset mhcflurry
Saving outputs to: iedb_afnd_pc_results
           peptide       allele locus
0        AADFPGIAR  HLA-A*01:01     A
1       AADKAAAAAY  HLA-A*01:01     A
2        AADKAAAAY  HLA-A*01:01     A
3        AADSFATSY  HLA-A*01:01     A
4       AAELDRTEEY  HLA-A*01:01     A
...            ...          ...   ...
187590   YVYFYDLSY  HLA-C*15:02     C
187591   YWMGGTTYF  HLA-C*15:02     C
187592   YYFSYPLFV  HLA-C*15:02     C
187593   YYGRWVHEF  HLA-C*15:02     C
187594   YYKKTFSAL  HLA-C*15:02     C

[187485 rows x 3 columns]
Looking at dataset mhcflurry with total 187485 peptides across 132 alleles 3 loci
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\mhcflurry_Australia_Groote_Eylandt_Aborigine_real.txt
python IEDB_Population_Coverage-3.0.2\population_coverage\calculate_population_coverage.py -p "Australia Groote Eylandt 

iedb_afnd_pc_results\hlathena_Australia_Groote_Eylandt_Aborigine_real.txt
Calculating PC for Australia Groote Eylandt Aborigine for ideal dataset hlathena
The original dataset hlathena has 142300 peptides
The population has 37 alleles across 3 loci
Ideal dataset has 142480 peptides across 33 alleles
    index   Allele Locus  ideal_n_peptides
0      23  A*01:01     A              1280
1      24  A*02:01     A              5075
2      25  A*11:01     A             11384
3      26  A*24:02     A             13897
4      29  A*31:01     A               332
5      30  A*32:01     A               332
6      31  A*34:01     A             15178
7   15145  B*08:01     B               948
8   15146  B*13:01     B             11051
9   15147  B*14:01     B               332
10  15148  B*15:02     B               332
11  15149  B*15:21     B              2846
12  15150  B*15:25     B              3794
13  15152  B*40:01     B              8205
14  15153  B*40:02     B              8538
15  15154  

...

iedb_afnd_pc_results\netmhc_United_Arab_Emirates_Abu_Dhabi_real.txt
Calculating PC for United Arab Emirates Abu Dhabi for ideal dataset netmhc
The original dataset netmhc has 170107 peptides
The population has 85 alleles across 3 loci
Ideal dataset has 170014 peptides across 85 alleles
    index   Allele Locus  ideal_n_peptides
0   14942  A*01:01     A              3816
1   14943  A*01:03     A               544
2   14944  A*02:01     A             11448
3   14945  A*02:05     A              2183
4   14946  A*02:06     A               544
..    ...      ...   ...               ...
80  50462  C*16:01     C               544
81  50463  C*16:02     C              3271
82  50464  C*17:01     C              1088
83  50465  C*17:03     C               544
84  50466  C*18:01     C               544

[85 rows x 4 columns]
Saved dummy ideal dataset to: iedb_afnd_pc_results\netmhc_United_Arab_Emirates_Abu_Dhabi_ideal.csv
Running IEDB tool for population coverage and saving output to: iedb_afnd_p

Plotting results
Calculating JS divergence
JS divergence: 0.7283005695218865
Calculating PC for Venezuela Sierra de Perija Yucpa for dataset hlathena
Saving outputs to: iedb_afnd_pc_results
             peptide       allele locus
0         AIDNSRNILY  HLA-A*01:01     A
1       AIGTYLDQYIVY  HLA-A*01:01     A
2          ALDDFTICY  HLA-A*01:01     A
3         ALDDFTICYF  HLA-A*01:01     A
4         ALDENGNLWY  HLA-A*01:01     A
...              ...          ...   ...
142295     YTSRIVVRL  HLA-C*17:01     C
142296     YVAPKAVWL  HLA-C*17:01     C
142297     YVAPRRILL  HLA-C*17:01     C
142298     YVVPYTIHL  HLA-C*17:01     C
142299     YVYKNVSQL  HLA-C*17:01     C

[142300 rows x 3 columns]
Looking at dataset hlathena with total 142300 peptides across 92 alleles 3 loci
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\hlathena_Venezuela_Sierra_de_Perija_Yucpa_real.txt
python IEDB_Population_Coverage-3.0.2\population_coverage\calculate_population_coverage

iedb_afnd_pc_results\hlathena_Vietnam_Hanoi_Kinh_pop_2_real.txt
Calculating PC for Vietnam Hanoi Kinh pop 2 for ideal dataset hlathena
The original dataset hlathena has 142300 peptides
The population has 76 alleles across 3 loci
Ideal dataset has 142502 peptides across 76 alleles
    index   Allele Locus  ideal_n_peptides
0   14975  A*01:01     A              1802
1   14976  A*02:01     A               996
2   14977  A*02:03     A              3747
3   14978  A*02:06     A              2229
4   14979  A*02:07     A              4031
..    ...      ...   ...               ...
71  50485  C*12:02     C               996
72  50486  C*12:03     C               426
73  50487  C*14:02     C              1233
74  50488  C*15:02     C              1375
75  50489  C*15:05     C              2371

[76 rows x 4 columns]
Saved dummy ideal dataset to: iedb_afnd_pc_results\hlathena_Vietnam_Hanoi_Kinh_pop_2_ideal.csv
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\

Plotting results
Calculating JS divergence
JS divergence: 0.5127701814412851
Calculating PC for Zambia Lusaka for dataset mhcflurry
Saving outputs to: iedb_afnd_pc_results
           peptide       allele locus
0        AADFPGIAR  HLA-A*01:01     A
1       AADKAAAAAY  HLA-A*01:01     A
2        AADKAAAAY  HLA-A*01:01     A
3        AADSFATSY  HLA-A*01:01     A
4       AAELDRTEEY  HLA-A*01:01     A
...            ...          ...   ...
187590   YVYFYDLSY  HLA-C*15:02     C
187591   YWMGGTTYF  HLA-C*15:02     C
187592   YYFSYPLFV  HLA-C*15:02     C
187593   YYGRWVHEF  HLA-C*15:02     C
187594   YYKKTFSAL  HLA-C*15:02     C

[187485 rows x 3 columns]
Looking at dataset mhcflurry with total 187485 peptides across 132 alleles 3 loci
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\mhcflurry_Zambia_Lusaka_real.txt
python IEDB_Population_Coverage-3.0.2\population_coverage\calculate_population_coverage.py -p "Zambia Lusaka" -c I -f C:\Users\ac121\Documents\HL

iedb_afnd_pc_results\mhcflurry_Zimbabwe_Harare_Shona_real.txt
Calculating PC for Zimbabwe Harare Shona for ideal dataset mhcflurry
The original dataset mhcflurry has 187485 peptides
The population has 91 alleles across 3 loci
Ideal dataset has 184423 peptides across 91 alleles
    index   Allele Locus  ideal_n_peptides
0   15069  A*01:01     A               249
1   15070  A*02:01     A              5687
2   15071  A*02:02     A              2249
3   15072  A*02:05     A              1937
4   15073  A*02:17     A               124
..    ...      ...   ...               ...
86  50555  C*14:03     C               249
87  50556  C*15:05     C               124
88  50557  C*16:01     C              4999
89  50558  C*17:01     C              5124
90  50559  C*18:01     C              3062

[91 rows x 4 columns]
Saved dummy ideal dataset to: iedb_afnd_pc_results\mhcflurry_Zimbabwe_Harare_Shona_ideal.csv
Running IEDB tool for population coverage and saving output to: iedb_afnd_pc_results\mhcfl